#### Import das bibliotecas que serão utilizadas e da base de dados da PCP

In [18]:
import pandas as pd

In [19]:
pcp = pd.read_excel(r"Ambiente-Modelo\Include\PCP Auto.xlsx", sheet_name=None)
print(pcp.keys())

dict_keys(['NCiv', 'NCon', 'NDados', 'NI', 'NTec', 'Projetos Internos', 'Respostas ao formulário 2'])


In [21]:
def nucleo_func(nucleo_digitado):
    nucleo_digitado = nucleo_digitado.replace(" ", "").lower()
    for pcp_sheet in pcp.keys():
        if nucleo_digitado == pcp_sheet.replace(" ", "").lower():
            pcp_nucleo = pcp[pcp_sheet]
            return pcp_nucleo
    return None

nucleo = input("Qual é o seu núcleo")
result = nucleo_func(nucleo)
print(result)

                Membro         Projeto 1 Início previsto Projeto 1  \
0     anaclara.esteves               HDI       2025-01-06 00:00:00   
1           mateus.ono                 -                         -   
2       matheus.arnold        Climatempo       2024-11-01 00:00:00   
3           artur.dias           Clealco       2024-04-01 00:00:00   
4         ayana.tanaka        Combate SF       2024-10-07 00:00:00   
5      natalia.martins                 -                         -   
6        natan.palacio            Priner       2024-10-21 00:00:00   
7        bruna.goulart                 -                         -   
8          caio.castro          Furukawa       2024-06-24 00:00:00   
9        pedro.beraldo                 -                         -   
10       pedro.cardoso                 -                         -   
11         pedro.porto  SHS Farmacêutica       2024-04-29 00:00:00   
12         diogo.kenzo        Climatempo       2024-11-01 00:00:00   
13         enzo.avel